In [1]:
import numpy as np 
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
#from scipy.interpolate import Akima1DInterpolator
#from scipy.stats import multivariate_normal
import time 
#import corner
import h5py
#import ipdb
from scipy import integrate 
from tqdm import tqdm
#import os
import pandas as pd
import camb 
from camb import model, initialpower
from CosmologyCalculatorPC import CosmologyCalculatorPC

# Import - define all necessary variables

In [2]:
# Redshifts for the bins that we chose when we did the eigevector analysis with Fisher.

Nx = 500
zzmin = 0.03
zzmax = 1.7
zBinsFisher = np.array([zzmin + i * (zzmax - zzmin) / Nx for i in range(1, Nx + 1)])
zbins = np.insert(zBinsFisher, 0, 0.03)

alphasfid = np.array([0] * 500) # When all amplitudes are zero, I get back ΛCDM

eigenvectors = np.loadtxt("/mnt/c/Users/asgia/Desktop/cosmo_stuff/eigenvectorsFisherTot_ver2_500PCs_0p03_to_1p7.dat")
#eigenvectors = np.loadtxt("/mnt/c/Users/asgia/Desktop/cosmo_stuff/eigenvectorsFisherTot_ver2.dat")

eigenvectors.shape

(500, 500)

In [3]:
# Returns fiducial ΛCDM distances
calculator = CosmologyCalculatorPC(73,0.24, 0, alphasfid, eigenvectors)

# Define the priors on the amplitudes $\alpha_i$ following Mortonson et al (0810.1744)

In [4]:
## In order to define the correct prior for the PC amplitudes, I need to define the allowed min and max values for each alpha_i.
## I can do that outside of the function that I will write below since these things do not change with each random step that the MCMC makes

# Theoretical limits on DE
wfid = -1
# Quintessence
wmin, wmax = -1, 1
# Smooth DE
# wmin, wmax = -5, 3
weight1 = wmin + wmax - 2 * wfid
weight2 = wmax - wmin 

# First I need to calculate each of the two relevant sums in the eq (A10)
first_sum = np.sum(eigenvectors, axis=1)
second_sum = np.sum(np.abs(eigenvectors), axis=1)

## Nz is the number of z bins I used 
Nz = 500

# alpha_max and alpha_min are two arrays that are storing the max and min value of the amplitudes alpha_i 
alpha_max = ( 1 / (2*Nz)) * ( weight1 * first_sum + weight2 * second_sum)
alpha_min = ( 1 / (2*Nz)) * ( weight1 * first_sum - weight2 * second_sum)

In [5]:
alpha_max[0], alpha_min[0]

(0.0, -0.04848137963350209)

- Define the limits of the sampling region for the amplitudes $\alpha_i$. At this point I also need to decide how many $\alpha_i$ I will keep.
- I will sample only for $\alpha_1$, $\Omega_m$ and $H_0$. I will set to zero the rest of the PCs. Also, note that based on the calculation in the previous cell, I know the min and max values that are allowed for $\alpha_i$.

In [6]:
##The fiducial model that I am using has Omega_m = 0.24 and h = 0.73 -- Omega_m h^2 = 0.127896 

### Example case for 50 PCs
low_lim = alpha_min[:50]
high_lim = alpha_max[:50]

low_lim = np.concatenate([alpha_min[:50],[69, 0.20 ]])
high_lim = np.concatenate([alpha_max[:50],[77, 0.28]])

In [7]:
np.random.uniform(low=low_lim, high=high_lim, size=None)

array([-1.95640092e-02,  5.85206683e-03,  1.57280008e-02,  2.74227913e-02,
        4.88169847e-03, -5.64003222e-03,  3.11905847e-02,  1.97014527e-02,
       -2.12351454e-02,  3.92108525e-02, -3.13021755e-02,  1.48569579e-02,
        6.91937203e-04, -1.57274871e-02,  1.10507235e-02,  2.07759894e-02,
        2.81199991e-02, -1.83153175e-02,  5.35965291e-04,  6.40526368e-03,
       -3.35962921e-02,  2.81060806e-02,  2.31260227e-02, -3.48900212e-02,
       -3.38464852e-02, -3.03115880e-02, -1.55265324e-02, -2.21245024e-02,
        1.19374384e-02,  3.23916108e-02,  3.45293107e-03,  1.01384812e-02,
       -2.91590606e-02, -2.74908379e-02,  2.86394872e-02,  1.68023843e-02,
       -8.42628532e-03, -2.19410544e-02,  8.05764453e-03,  2.95762578e-03,
        3.08140639e-02, -9.54529722e-03, -2.73772501e-03,  3.27178129e-02,
        1.23699532e-02, -1.78790427e-02,  1.70100514e-04, -2.80490477e-02,
        2.86926458e-02, -3.40399822e-02,  7.59676885e+01,  2.20834960e-01])

### Generate the cosmological parameters vectors 

In [8]:
# Will save the uniform sampled cosmological parameters in an hdf5 file

# with h5py.File('/gpfs/scratch/argiannakopo/uniform_cosmo_data_15PC.h5', 'w') as f:

with h5py.File('/home/venus/cosmo_temp/uniform_sampling_res/uniform_cosmo_params_50PC.h5', 'w') as f:
    # Will save things in chunks so that it is easier to load and read later
    dataset = f.create_dataset('data', shape=(0, 52), maxshape=(None, 52), dtype=np.float64, chunks=True)

    chunk_size = 10000  # Number of rows per chunk
    for start in range(0, 100000, chunk_size):
        end = min(start + chunk_size, 100000)
        # Uniformly distributed values for the amplitudes α_i
        chunk = np.random.uniform(low=low_lim, high=high_lim, size=(end - start, 52))
        
        dataset.resize((dataset.shape[0] + chunk.shape[0]), axis=0)
        dataset[-chunk.shape[0]:] = chunk

### Import the cosmological parameter vectors from file and calculate the equivalent luminosity distance

In [9]:
# Input and output file names
# input_filename = '/gpfs/scratch/argiannakopo/uniform_cosmo_data_15PC.h5'
# output_filename = '/gpfs/scratch/argiannakopo/uniform_lodL_data_15PC.h5'

input_filename = '/home/venus/cosmo_temp/uniform_sampling_res/uniform_cosmo_params_50PC.h5'
output_filename = '/home/venus/cosmo_temp/uniform_sampling_res/uniform_logdl_vals_50PC.h5'

zeros = [0] * (500 - 50)

# Read the input file and process each row
with h5py.File(input_filename, 'r') as f_in, h5py.File(output_filename, 'w') as f_out:
    dataset_in = f_in['data']
    num_rows, num_cols = dataset_in.shape
    
    
    dataset_out = f_out.create_dataset('processed_data', shape=(num_rows, 500), dtype=np.float64, chunks=True)
    
    # Process each row and save the result
    for i in tqdm(range(num_rows)):
        row = dataset_in[i]
        alps = np.concatenate([row[:50],zeros])  
        calculator50 = CosmologyCalculatorPC(row[-2], row[-1], 0, alps, eigenvectors)
        processed_row = calculator50.lum_distance(zbins, zBinsFisher)
        dataset_out[i] = np.log((row[-2] / 100) * processed_row)

print("Processing complete. Data saved to:", output_filename)


100%|██████████████████████████████████████████████████████████████████████████| 100000/100000 [05:35<00:00, 297.90it/s]


Processing complete. Data saved to: /home/venus/cosmo_temp/uniform_sampling_res/uniform_logdl_vals_50PC.h5
